In [1]:
import torch
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from skimage.measure import marching_cubes
import sys
import os

sys.path.append(os.getcwd())
from utils.FragmentDataset import FragmentDataset, ToTensor3D
from utils.model import Generator


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT_PATH = 'checkpoints/best_model_400.pth' 
DATA_DIR = './data'
THRESHOLD = 0.5 

def get_mesh_trace(voxel_grid, color, name, opacity=1.0):
    """
    使用 Marching Cubes 算法将二值体素转化为 3D 网格表面
    """
    # Marching cubes 需要 volume 数据，且最好稍微 pad 一下防止边缘截断
    padded_grid = np.pad(voxel_grid, 1, mode='constant', constant_values=0)
    
    try:
        # 计算顶点和面
        verts, faces, normals, values = marching_cubes(padded_grid, level=0.5)
        # 因为 pad 了一圈，坐标要减 1 还原
        verts = verts - 1
        
        mesh = go.Mesh3d(
            x=verts[:, 0], 
            y=verts[:, 1], 
            z=verts[:, 2],
            i=faces[:, 0], 
            j=faces[:, 1], 
            k=faces[:, 2],
            color=color,
            opacity=opacity,
            name=name,
            flatshading=False, # 开启平滑着色
            lighting=dict(
                ambient=0.4,
                diffuse=0.5,
                roughness=0.1,
                specular=0.4,
                fresnel=0.2
            ),
            lightposition=dict(x=100, y=100, z=500)
        )
        return mesh
    except RuntimeError:
        # 如果物体是空的或者太碎无法生成 Mesh，回退到散点图
        print(f"Warning: Could not generate mesh for {name}, falling back to scatter.")
        return get_scatter_trace(voxel_grid, color, name, opacity)

def get_scatter_trace(voxel_grid, color, name, opacity=0.5, size=2):
    """
    回退方案：绘制散点，但优化点的大小和透明度
    """
    x, y, z = np.where(voxel_grid > 0)
    return go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        name=name,
        marker=dict(
            size=size,
            symbol='square',
            color=color,
            opacity=opacity,
            line=dict(width=0) # 关键：去掉轮廓线，防止变黑
        )
    )


def visualize_sample_mesh(dataset, index=None, model=None, class_index=None):

    sample_index = [0, 2065, 2272, 2593, 2640, 2896, 3136, 3360, 4656, 128, 1600] # 每个类别的一个样本索引
    if index is None:
        if class_index is None:
            index = np.random.randint(0, len(dataset))
        else:
            index = sample_index[class_index]
    
    frag, gt, label, path = dataset[index]
    
    frag_tensor = frag.unsqueeze(0).to(DEVICE) # (1, 4, 64, 64, 64)
    label_tensor = torch.tensor([label]).to(DEVICE)
    
    model.eval()
    with torch.no_grad():
        output = model(frag_tensor, label_tensor)
        # 二值化
        pred_occ = (output.squeeze().cpu().numpy() > THRESHOLD).astype(np.uint8)
    
    # 提取原始 numpy 数据
    input_occ = frag[0].numpy().astype(np.uint8) # Channel 0 是 Occupancy
    gt_occ = gt.squeeze().numpy().astype(np.uint8)
    
    print(f"Visualizing Index: {index} | Class: {label}")
    print(f"Path: {path}")

    # 创建 Plotly 画布 (1行3列)
    fig = make_subplots(
        rows=1, cols=3,
        specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=('Input Fragment (Points)', 'Reconstruction (Mesh)', 'Ground Truth (Mesh)')
    )

    # --- Subplot 1: Input (输入一般比较稀疏，用点云看更直观，也可以用Mesh) ---
    # 这里我们用半透明的点，方便看内部
    trace_input = get_scatter_trace(input_occ, 'gray', 'Input', opacity=0.6, size=3)
    fig.add_trace(trace_input, row=1, col=1)

    # --- Subplot 2: Prediction (Mesh) + Input (Points) ---
    # 用红色 Mesh 展示生成的形状，同时把灰色的输入点画在里面，展示补全关系
    trace_pred_mesh = get_mesh_trace(pred_occ, '#d05d86', 'Prediction', opacity=1.0) # 陶土红
    fig.add_trace(trace_pred_mesh, row=1, col=2)
    # 叠加输入点（可选，有助于看出哪里是补全的）
    # fig.add_trace(get_scatter_trace(input_occ, 'black', 'Input Ref', opacity=0.3, size=2), row=1, col=2)

    # --- Subplot 3: Ground Truth (Mesh) ---
    trace_gt_mesh = get_mesh_trace(gt_occ, '#2992BC', 'Ground Truth', opacity=1.0) # 青蓝色
    fig.add_trace(trace_gt_mesh, row=1, col=3)

    # 调整相机和布局
    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.5)
    )
    
    fig.update_layout(
        height=600, width=1200,
        title_text=f"3D Surface Reconstruction (Index: {index}, Class:{label})",
        scene=dict(camera=camera, aspectmode='data'),
        scene2=dict(camera=camera, aspectmode='data'),
        scene3=dict(camera=camera, aspectmode='data')
    )
    
    fig.show()

# 重新加载 dataset 和 model
print("Loading Test Dataset...")
test_set = FragmentDataset(DATA_DIR, 'test', transform=ToTensor3D(binary=True))


print(f"Loading Model from {CHECKPOINT_PATH}...")
model = Generator(in_channels=4).to(DEVICE)
if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    state_dict = checkpoint['G_state_dict']
    new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(new_state_dict)
    print("Model loaded.")
else:
    print("Checkpoint not found!")

visualize_sample_mesh(test_set, model=model)

# for i in range(11):
#     print(f"Visualizing class index: {i}")
#     visualize_sample_mesh(test_set, model=model, class_index=i)  # 0: Chair

Loading Test Dataset...
[TEST] Dataset initialized. Found 4752 files.
Loading Model from checkpoints/best_model_400.pth...
Model loaded.
Visualizing Index: 4205 | Class: 7
Path: ./data/test/8/CC_24_1-n016-t1649438884.vox
